In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix

import itertools
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
train = pd.read_table('train/subsidy_train.txt',sep=',',header=-1)
train.columns = ['id','label']
test = pd.read_table('test/studentID_test.txt',sep=',',header=-1)
test.columns = ['id']
test['label'] = np.nan

#train_test = pd.concat([train,test])
train_test=train

del train
del test

'''
score_train = pd.read_table('train/score_train.txt',sep=',',header=-1)
score_train.columns = ['id','college','rank']
score_test = pd.read_table('test/score_test.txt',sep=',',header=-1)
score_test.columns = ['id','college','rank']
score_train_test = pd.concat([score_train,score_test])

college = pd.DataFrame(score_train_test.groupby(['college'])['rank'].max())
college.to_csv('input/college.csv',index=True)
college = pd.read_csv('input/college.csv')
college.columns = ['college','total_people']

score_train_test = pd.merge(score_train_test, college, how='left',on='college')
score_train_test['rank_percent'] = score_train_test['rank']/score_train_test['total_people']
train_test = pd.merge(train_test,score_train_test,how='left',on='id')
'''

card_train = pd.read_table('train/card_train.txt',sep=',',header=-1)
card_train.columns = ['id','pos','place','consume','time','price','rest']
#card_test = pd.read_table('test/card_test.txt',sep=',',header=-1)
#card_test.columns = ['id','pos','place','consume','time','price','rest']
#
#card_train_test = pd.concat([card_train,card_test])
card_train_test=card_train
print "Read OK!"

##release memery
del card_train
#del card_test

Read OK!


In [3]:
#card_train_test=card_train_test.drop_duplicates()

In [4]:
#card_train_test.drop(['place','time'],axis=1,inplace=True)

In [5]:
card_train_test.shape

(12455558, 7)

In [6]:
card_train_test.head()

,id,pos,place,consume,time,price,rest
0,1006,POS消费,地点551,淋浴,2013/09/01 00:00:32,0.50,124.90
1,1006,POS消费,地点551,淋浴,2013/09/01 00:00:32,0.50,124.90
2,1968,POS消费,地点159,淋浴,2013/09/01 00:00:39,0.10,200.14
3,1968,POS消费,地点159,淋浴,2013/09/01 00:00:39,0.10,200.14
4,1406,POS消费,地点660,开水,2013/09/01 00:00:40,0.01,374.42


In [7]:
card_train_test['place'].value_counts().index

Index([u'地点21', u'地点91', u'地点232', u'地点188', u'地点192', u'地点118', u'地点26',
       u'地点996', u'地点6', u'地点83',
       ...
       u'地点1412', u'地点2332', u'地点2696', u'地点2697', u'地点2692', u'地点2693',
       u'地点2691', u'地点2699', u'地点2686', u'地点2694'],
      dtype='object', length=2468)

In [8]:
place_info=card_train_test['place'].value_counts()

In [9]:
place_more=place_info[place_info>5000]

In [10]:
for var in place_more.index:
    place = card_train_test[card_train_test.place == var]
    
    feature_p = pd.DataFrame(place.groupby(['id'])['pos'].count())
    feature_p['%s_sum'%var]=place.groupby(['id'])['price'].sum()
    feature_p['%s_avg'%var]=place.groupby(['id'])['price'].mean()
    feature_p['%s_max'%var]=place.groupby(['id'])['price'].max()
    feature_p['%s_min'%var]=place.groupby(['id'])['price'].min()
    feature_p['%s_median'%var]=place.groupby(['id'])['price'].median()
    
    del place
    feature_p.to_csv('input/card_%sfeature.csv'%var,index=True)
    feature_p=pd.read_csv('input/card_%sfeature.csv'%var)
    feature_p=feature_p.rename(columns={'pos' : '%s_count'%var})
    
    train_test = pd.merge(train_test, feature_p, how='left',on='id') 
    del feature_p

In [11]:
#del card_train_test
train_test=train_test.fillna(-1)

In [12]:
train_test.head()

,id,label,地点21_count,地点21_sum,地点21_avg,地点21_max,地点21_min,地点21_median,地点91_count,地点91_sum,...,地点145_avg,地点145_max,地点145_min,地点145_median,地点355_count,地点355_sum,地点355_avg,地点355_max,地点355_min,地点355_median
0,0,0,94.0,421.50,4.484043,23.1,0.1,3.0,67.0,378.49,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
1,1,0,66.0,260.42,3.945758,20.0,0.1,3.0,14.0,72.40,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
2,8,0,651.0,2424.96,3.724977,31.5,-5.0,3.1,262.0,2203.45,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,9,0,1047.0,4741.29,4.528453,37.0,0.2,3.0,25.0,142.10,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,10,0,121.0,747.43,6.177107,30.0,-6.8,5.1,88.0,445.30,...,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [13]:
## Evaluation 
def f1_macro(label_truth, predictions):
    df=pd.DataFrame(columns=["subsidy_x","subsidy_y"])
    df.subsidy_y=predictions
    df.subsidy_x=np.array(label_truth)
    df.subsidy_y = df.subsidy_y.apply(lambda x:int(x))

    
    correct = df[df['subsidy_x'] == df['subsidy_y']]
    s = 0
    for i in [1000, 1500, 2000]:
        #print i
        r = float(sum(correct['subsidy_y'] == i))/sum(df['subsidy_x'] == i)
        #print r
        p = float(sum(correct['subsidy_y'] == i))/sum(df['subsidy_y'] == i)        
        #print p
        f = r*p*2/(r+p)
        if not np.isnan(f):
            s += (float(sum(df['subsidy_x'] == i))/df.shape[0])*f
    return s

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [116]:
X = train_test.ix[:, train_test.columns != 'label']
y = train_test.ix[:, train_test.columns == 'label']

In [117]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.3, random_state = 42)

In [118]:
train=X_train
train['label']=y_train

test=X_test

/home/kuhung/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [130]:
#nice_feature=pd.read_csv('input/nice_feature.csv',header=None,index_col=0)

target = 'label'
IDcol = 'id'
ids = test['id'].values

all_feature = [x for x in train.columns if x not in [target]]
predictors = [ x for x in all_feature if x in feature_imp_place20.feature.values]
#predictors = [ x for x in all_feature if x in nice_feature.index]

In [131]:
predictors_noID = [x for x in predictors if x != 'id']

In [121]:
y_train['label'].value_counts()

0       6538
1000     524
1500     308
2000     249
Name: label, dtype: int64

In [122]:

# tips：
# 尝试欠采样和smoth采样

# undersample
train.to_csv('input/train_time-part.csv',index=False)
train=pd.read_csv('input/train_time-part.csv')

number_records_fraud = len(train[train.label != 0])
fraud_indices = np.array(train[train.label != 0].index)

# Picking the indices of the normal classes
normal_indices = train[train.label == 0].index

# Out of the indices we picked, randomly select "x" number (number_records_fraud)
random_normal_indices = np.random.choice(normal_indices, number_records_fraud, replace = False)
random_normal_indices = np.array(random_normal_indices)

# Appending the 2 indices
under_sample_indices = np.concatenate([fraud_indices,random_normal_indices])

# Under sample dataset
under_sample_data = train.iloc[under_sample_indices,:]

X_undersample = under_sample_data.ix[:, under_sample_data.columns != 'label']
y_undersample = under_sample_data.ix[:, under_sample_data.columns == 'label']

# Showing ratio
print("Number of normal transactions: ", len(under_sample_data[under_sample_data.label == 0]))
print("Number of fraud transactions: ", len(under_sample_data[under_sample_data.label != 0]))
print("Total number of transactions in resampled data: ", len(under_sample_data))


# Oversample
Oversampling1000 = train.loc[train.label == 1000]
Oversampling1500 = train.loc[train.label == 1500]
Oversampling2000 = train.loc[train.label == 2000]
'''
for i in range(6):
    train = train.append(Oversampling1000)
for j in range(10):
    train = train.append(Oversampling1500)
for k in range(12):
    train = train.append(Oversampling2000)
'''
for i in range(5):
    train = train.append(Oversampling1000)
for j in range(8):
    train = train.append(Oversampling1500)
for k in range(10):
    train = train.append(Oversampling2000)
    
train = train[train['label'].notnull()]

('Number of normal transactions: ', 1081)
('Number of fraud transactions: ', 1081)
('Total number of transactions in resampled data: ', 2162)


In [123]:
train.shape

(15193, 1820)

In [133]:
# model

clf=XGBClassifier(seed=42)
#clf = RandomForestClassifier()
#clf = GradientBoostingClassifier(random_state=42,n_estimators=200)
#clf = clf.fit(np.log(X_undersample[predictors]),y_undersample[target])

clf = clf.fit(np.log(train[predictors_noID]),train[target])

result = clf.predict(np.log(test[predictors_noID]))
f1_macro(y_test,result)

0.02564783038258662

In [80]:
feat_imp=pd.Series(clf.booster().get_fscore()).sort_values(ascending=False)

In [102]:
feature_imp_place20=feat_imp.head(20)

In [103]:
feature_imp_place20

地点21_count      83
地点21_avg        47
地点829_sum       32
地点818_max       19
地点829_min       18
地点213_avg       16
地点72_count      16
地点283_count     16
地点91_count      16
地点245_avg       15
地点283_sum       15
地点213_count     15
地点65_max        15
地点161_sum       15
地点996_count     15
地点277_median    14
地点842_max       14
地点75_max        14
地点263_avg       14
地点840_count     14
地点227_sum       14
地点21_min        14
地点844_count     13
地点268_max       13
地点72_min        13
地点996_avg       13
地点272_sum       12
地点232_median    12
地点21_max        12
地点26_min        12
dtype: int64

In [105]:
feature_imp_place20.index

Index([u'地点21_count', u'地点21_avg', u'地点829_sum', u'地点818_max', u'地点829_min',
       u'地点213_avg', u'地点72_count', u'地点283_count', u'地点91_count',
       u'地点245_avg', u'地点283_sum', u'地点213_count', u'地点65_max', u'地点161_sum',
       u'地点996_count', u'地点277_median', u'地点842_max', u'地点75_max',
       u'地点263_avg', u'地点840_count', u'地点227_sum', u'地点21_min', u'地点844_count',
       u'地点268_max', u'地点72_min', u'地点996_avg', u'地点272_sum', u'地点232_median',
       u'地点21_max', u'地点26_min'],
      dtype='object')

In [ ]:
feature_imp_place20=feature_imp_place20.index

In [ ]:
'''
## 这里的编码方式待处理，否则无法输出。简单起见，我把以下feature复制出去。
feature_imp_place20.to_csv('input/feature_imp_place30.csv')
'''

In [128]:
feature_imp_place20=pd.read_csv('input/feature_imp_place20.csv')

In [129]:
feature_imp_place20.feature.values

array(['\xe5\x9c\xb0\xe7\x82\xb921_count',
       '\xe5\x9c\xb0\xe7\x82\xb921_avg', '\xe5\x9c\xb0\xe7\x82\xb9829_sum',
       '\xe5\x9c\xb0\xe7\x82\xb9818_max',
       '\xe5\x9c\xb0\xe7\x82\xb9829_min',
       '\xe5\x9c\xb0\xe7\x82\xb9213_avg',
       '\xe5\x9c\xb0\xe7\x82\xb972_count',
       '\xe5\x9c\xb0\xe7\x82\xb9283_count',
       '\xe5\x9c\xb0\xe7\x82\xb991_count',
       '\xe5\x9c\xb0\xe7\x82\xb9245_avg',
       '\xe5\x9c\xb0\xe7\x82\xb9283_sum',
       '\xe5\x9c\xb0\xe7\x82\xb9213_count',
       '\xe5\x9c\xb0\xe7\x82\xb965_max', '\xe5\x9c\xb0\xe7\x82\xb9161_sum',
       '\xe5\x9c\xb0\xe7\x82\xb9996_count',
       '\xe5\x9c\xb0\xe7\x82\xb9277_median',
       '\xe5\x9c\xb0\xe7\x82\xb9842_max', '\xe5\x9c\xb0\xe7\x82\xb975_max',
       '\xe5\x9c\xb0\xe7\x82\xb9263_avg',
       '\xe5\x9c\xb0\xe7\x82\xb9840_count'], dtype=object)